In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import os
import itertools
import sys
sys.path.append('/home/lijunzhang/multibranch/')
from pathlib import Path
import random
from scipy import stats
from ptflops import get_model_complexity_info
import matplotlib.pyplot as plt

from main.algs_EstCon import *
from main.layout import Layout
from main.algorithms import enum_layout_wo_rdt, init_S, coarse_to_fined

# Est. Rank

In [2]:
# hyper
exp_dir = 'FastMTL_0508'

load_weight = False
short_iters = 200
smooth_weight = 0.0

data = 'NYUv2'
backbone = 'mobilenet'

folder = '_'.join([str(load_weight), str(short_iters), str(smooth_weight)])+'/'
savepath = os.path.join('./ckpt/', exp_dir, folder)

In [3]:
if os.path.exists(os.path.join(savepath, 'layout_est.pkl')):
    layout_est = load_obj(savepath, 'layout_est')
else:
    layout_est = None
    
if data == 'NYUv2':
    tasks = ['segment_semantic', 'normal', 'depth_zbuffer']
elif args.data == 'Taskonomy':
    tasks = ['segment_semantic','normal','depth_zbuffer','keypoints2d','edge_texture']
T = len(tasks)

if backbone == 'resnet34':
    D = coarse_B = 5
elif backbone == 'mobilenet':
    D = coarse_B = 5

In [4]:
def check_valid(result, tasks):
    alpha_dict = result['alpha']
    for task in tasks:
        if len(alpha_dict[task]) == 0:
            return False
    return True

def extract_result(layout_est, savepath, layout_idx):
    valid = False
    # extract from the overall results
    if layout_est is not None:
        result = layout_est[layout_idx]
        if check_valid(result, tasks):
            valid = True
    # extract from the single result
    if layout_est is None or not valid:
        if os.path.exists(os.path.join(savepath, 'layout_'+str(layout_idx)+'.pkl')):
            result = load_obj(savepath, 'layout_'+str(layout_idx))
            if not check_valid(result, tasks):
                return False
        else: # single result not exists
            return False
    return result

In [5]:
# enum layout
layout_list = [] 
S0 = init_S(T, coarse_B) # initial state
L = Layout(T, coarse_B, S0) # initial layout
layout_list.append(L)
enum_layout_wo_rdt(L, layout_list)
print('Finish Layout Enumeration', flush=True)

layout_idx = -1
rerun = []
outlier = False
# take down conv_iter and virtual_loss for each layout
for L in layout_list:
    layout_idx += 1
    if layout_idx == 2:
        continue
    
    result = extract_result(layout_est, savepath, layout_idx)
    if not result:
        print('Please Rerun Layout {}.'.format(layout_idx),flush=True)
        rerun.append(layout_idx)
        continue
    
    for key in result:
        if key == 'alpha':
            continue
        content = result[key]
        for task in tasks:
            re_lst = content[task]
            if outlier:
                # IQR
                Q1 = np.percentile(re_lst, 25, interpolation = 'midpoint')
                Q3 = np.percentile(re_lst, 75, interpolation = 'midpoint')
                IQR = Q3 - Q1
                upper = Q3+1.5*IQR # Upper bound
                lower = Q1-1.5*IQR # Lower bound
                re_lst = [x for x in re_lst if lower<=x and x<=upper]
            
            if len(re_lst) > 0:
                if key == 'est_iter':
                    L.conv_iter.append((min(re_lst),max(re_lst)))
                elif key == 'est_loss':
                    L.virtual_loss.append(np.mean(re_lst))
            else:
                print('Please Rerun Layout {}.'.format(layout_idx),flush=True)
                rerun.append(layout_idx)

Finish Layout Enumeration


In [6]:
final_loss_flag = True # true: rank by final loss; false: rank by conv iter

if final_loss_flag:
    # sort by final loss
    layout_order = sorted(range(len(layout_list)), key=lambda k: np.sum(layout_list[k].virtual_loss),reverse=False)
    for idx in layout_order:
        print('Layout Idx: {}'.format(idx))
        print('Virtual Loss: {}'.format(sum(layout_list[idx].virtual_loss)))
        print('-'*80)
else:
    # sort by conv iter
    layout_order = sorted(range(30), key=lambda k: int(np.max(np.mean(layout_list[k].conv_iter, axis=1))),reverse=False)
    for idx in layout_order:
        print('Layout Idx: {}'.format(idx))
        print('Conv Iter: {}'.format(int(np.max(np.mean(layout_list[idx].conv_iter, axis=1)))))
        print('Confidence: {}'.format(200*1/np.mean([layout_list[idx].conv_iter[i][1]-layout_list[idx].conv_iter[i][0] for i in range(T)])))
        print('-'*80)

Layout Idx: 16
Virtual Loss: -97.9358394626398
--------------------------------------------------------------------------------
Layout Idx: 10
Virtual Loss: -93.48567664557919
--------------------------------------------------------------------------------
Layout Idx: 21
Virtual Loss: -90.8198577669443
--------------------------------------------------------------------------------
Layout Idx: 22
Virtual Loss: -90.00532094025623
--------------------------------------------------------------------------------
Layout Idx: 39
Virtual Loss: -88.09674355893787
--------------------------------------------------------------------------------
Layout Idx: 28
Virtual Loss: -87.41130304779801
--------------------------------------------------------------------------------
Layout Idx: 13
Virtual Loss: -86.59257596343585
--------------------------------------------------------------------------------
Layout Idx: 41
Virtual Loss: -86.11674411287493
---------------------------------------------------

# SROCC

In [7]:
real_loss_conv_iter = load_obj('./exp/', 'real_loss_conv_iter_'+data+'_'+backbone)
real_early_iter_rel_pref = load_obj('./exp/', 'real_early_iter_rel_pref_'+data+'_'+backbone)

idx = real_loss_conv_iter['layout']
est_final_loss = [sum(layout_list[i].virtual_loss) for i in idx]
est_conv_iter = [int(np.max(np.mean(layout_list[i].conv_iter, axis=1))) for i in idx]

In [11]:
real_final_loss = real_loss_conv_iter['final_loss']
real_conv_iter = real_loss_conv_iter['conv_iter']

real_early_iter = real_early_iter_rel_pref['early_iter']
real_rel_pref = real_early_iter_rel_pref['final_rel_perf']

In [12]:
stats.spearmanr(real_final_loss,est_final_loss)

SpearmanrResult(correlation=0.312781954887218, pvalue=0.17936353052759138)

In [13]:
stats.spearmanr(real_conv_iter,est_conv_iter)

SpearmanrResult(correlation=0.15851832887861342, pvalue=0.5044468512716638)

In [14]:
stats.spearmanr(real_rel_pref,est_final_loss)

SpearmanrResult(correlation=-0.17744360902255638, pvalue=0.4542034838793192)

In [15]:
stats.spearmanr(real_early_iter,est_conv_iter)

SpearmanrResult(correlation=0.07214103567702387, pvalue=0.7624654621399827)

In [16]:
stats.spearmanr(real_final_loss,real_rel_pref) #????

SpearmanrResult(correlation=-0.7278195488721804, pvalue=0.00027529626409060167)

In [17]:
stats.spearmanr(real_conv_iter,real_early_iter)

SpearmanrResult(correlation=0.3025415688536524, pvalue=0.19479436311346848)